In [127]:
import pickle
import pandas as pd
import nltk
import pycountry
import string
import re

In [128]:
df = pd.read_pickle('deception_df.pkl')
df_2 = pd.read_json('diplomacy_data.json')

In [129]:
df.sample(3)

,messages,sender_labels,receiver_labels,speakers,receivers,absolute_message_index,relative_message_index,seasons,years,game_score,game_score_delta,players,game_id,politeness,negative_sentiment,neutral_sentiment,positive_sentiment,vader_score
5606,I managed to piss Germany off for you 😉,True,True,austria,england,2675,95,Spring,1905,8,2,"austria,england",2,2.220411,1.0,0.0,0.0,-0.4019
1350,"Oh fair, because you're not actually moving in...",True,True,england,germany,407,61,Spring,1902,4,-1,"germany,england",1,3.327725,0.4,0.6,0.0,0.4019
13187,You stabbing the French is already a massive l...,True,True,italy,germany,1007,55,Winter,1904,3,-6,"italy,germany",11,3.150316,0.5,0.5,0.0,-0.3412


In [130]:
df.describe()

,absolute_message_index,relative_message_index,game_id,politeness,negative_sentiment,neutral_sentiment,positive_sentiment,vader_score
count,17289.000000,17289.000000,17289.000000,17289.000000,17289.000000,17289.000000,17289.000000,17289.000000
mean,948.323153,113.711782,5.401990,3.335990,0.254738,0.605296,0.139966,0.195610
std,750.710891,129.367961,3.778135,0.333862,0.376942,0.420526,0.303795,0.413835
min,0.000000,0.000000,1.000000,1.349828,0.000000,0.000000,0.000000,-0.973100
25%,360.000000,22.000000,2.000000,3.168759,0.000000,0.000000,0.000000,0.000000
50%,769.000000,65.000000,4.000000,3.297227,0.000000,0.666667,0.000000,0.168000
75%,1347.000000,152.000000,9.000000,3.461842,0.500000,1.000000,0.000000,0.510600
max,3301.000000,674.000000,12.000000,4.588427,1.000000,1.000000,1.000000,0.991600


### 1. Looking at some messages for some politeness score slices (start, first quartile, third quartile, end)

* #### Looking at least polite messages

In [131]:
for _, row in df.sort_values('politeness').head(5).iterrows():
    print(f"Score: {row['politeness']:.3f} | Message: {row['messages']}")

Score: 1.350 | Message: I will not fuck you over
Score: 1.422 | Message: What a clod you are. How in blue fuck does keeping Turkey in the game benefit you in the least.
Score: 1.449 | Message: Don’t fuck me over tho
Score: 1.468 | Message: You're a dick dude.  I will make sure that that's the last one you take.  I'll let E/G walk over me of I have to
Score: 1.477 | Message: I was getting you into Kiel. You made a bad decision, you ruined my game, and you can go fuck yourself.


* #### Looking at first politeness quartile messages

In [132]:
for _, row in df.sort_values('politeness').iloc[int(len(df)/4):,:].head(5).iterrows():
    print(f"Score: {row['politeness']:.3f} | Message: {row['messages']}")

Score: 3.169 | Message: Totally understand that, I don’t want to be allied with the Turks but warring them and Russia at the same time is a pain in the ass. I’m convinced that they are against Russia and will help me until they know russia will go down (which could very well only be this fall or fall and spring)
Score: 3.169 | Message: He’s a little salty with me for the Belgium thing
Score: 3.169 | Message: Okay I’m reading your proposal again and are you saying you would get all of russia?
Score: 3.169 | Message: If France is honest with me, Ruhr will be free and Bur will be occupied. 

Let's be real here, though: Russia has at most two units to spare to threaten you, and likely not even that given the imminent pressure from Austria. I'm trying to work it out in the sandbox now but I really don't think they'd be an issue for you...
Score: 3.169 | Message: Either I build armies and funnel through Scandinavia to help you or build fleets and take on Italy.


* #### Looking at last politeness quartile messages

In [133]:
for _, row in df.sort_values('politeness').iloc[int(3*len(df)/4):,:].head(5).iterrows():
    print(f"Score: {row['politeness']:.3f} | Message: {row['messages']}")

Score: 3.462 | Message: Yeah, I think you'll be fine with Austria initially
Score: 3.462 | Message: You are correct, and that's why I want the army to go down there to help out silly head. Not sure where the confusion stems from.
Score: 3.462 | Message: Yeah I was thinking actually that it would make sense to put Russia into munich
Score: 3.462 | Message: Hey Italy, everything well over here. We probably won't be directly involved with each-other until mid-game, but if we're both still around, a France/Italy alliance could be pretty strong😉 .
Score: 3.462 | Message: Yeah prob either south coast or army. Im thinking an army could be good since I already have one fleet there


* #### Looking at most polite messages

In [134]:
for _, row in df.sort_values('politeness', ascending=False).head(5).iterrows():
    print(f"Score: {row['politeness']:.3f} | Message: {row['messages']}")

Score: 4.588 | Message: Thanks, much appreciated - Hoping to work together more going forward!
Score: 4.584 | Message: Ok, thank you!
Score: 4.567 | Message: That would be great! Thanks!
Score: 4.566 | Message: Okay, thank you for letting me know. That helps!
Score: 4.554 | Message: Sounds great man! Thanks for everything!


### 2. Looking at term frequency for different politeness slices 

* #### least polite messages

we take only 100 messages here to really be able to study extremely unpolite messages

In [135]:
# check for least polite vocabulary  


top_N = 10
txt = df.sort_values('politeness').head(100).messages.str.lower().str.replace(r'\|', ' ').str.cat(sep=' ')
txt = txt.translate(str.maketrans('', '', string.punctuation+'’´'))


pattern = re.compile(names, re.UNICODE | re.IGNORECASE)
words = nltk.tokenize.word_tokenize(txt)

stopwords = nltk.corpus.stopwords.words('english')
words_dist = nltk.FreqDist(w for w in words if w not in stopwords+['england','russia','france','turkey','germany','italy','austria-hungary', 'austria'])

rslt = pd.DataFrame(words_dist.most_common(top_N),
                    columns=['Word', 'Frequency'])
display(rslt)

,Word,Frequency
0,im,29
1,fuck,27
2,fucking,15
3,shit,14
4,game,11
5,fuckin,9
6,going,9
7,dont,8
8,two,8
9,army,8


* #### beginning of first quartile vocabulary

studying the quartiles we are not in the extremes anymore and can therefore take more messages

In [136]:
# first quartile vocabulary distribution

top_N = 10
txt = df.sort_values('politeness').iloc[int(len(df)/4):,:].head(1000).messages.str.lower().str.replace(r'\|', ' ').str.cat(sep=' ')
txt = txt.translate(str.maketrans('', '', string.punctuation+'’´'))
words = nltk.tokenize.word_tokenize(txt)

words_dist = nltk.FreqDist(w for w in words if w not in stopwords+['england','russia','france','turkey','germany','italy','austria-hungary', 'austria'])

rslt = pd.DataFrame(words_dist.most_common(top_N),
                    columns=['Word', 'Frequency'])
display(rslt)

,Word,Frequency
0,im,121
1,support,87
2,move,84
3,get,83
4,think,78
5,would,73
6,dont,72
7,going,67
8,like,60
9,want,58


* #### beginning of last quartile vocabulary

In [137]:
#third quartile
top_N = 10
txt = df.sort_values('politeness').iloc[int(3*len(df)/4):,:].head(1000).messages.str.lower().str.replace(r'\|', ' ').str.cat(sep=' ')
txt = txt.translate(str.maketrans('', '', string.punctuation+'’´'))
words = nltk.tokenize.word_tokenize(txt)

words_dist = nltk.FreqDist(w for w in words if w not in stopwords+['england','russia','france','turkey','germany','italy','austria-hungary', 'austria'])

rslt = pd.DataFrame(words_dist.most_common(top_N),
                    columns=['Word', 'Frequency'])
display(rslt)



,Word,Frequency
0,im,227
1,would,177
2,think,174
3,like,154
4,get,125
5,support,123
6,well,118
7,move,96
8,yeah,88
9,ill,88


* #### most polite messages vocabulary

we take only 100 messages here to really be able to study extremely polite messages

In [138]:
# Check for most polite vocabulary
txt = df.sort_values('politeness', ascending=False).head(100).messages.str.lower().str.replace(r'\|', ' ').str.cat(sep=' ')
txt = txt.translate(str.maketrans('', '', string.punctuation+'’´'))
words = nltk.tokenize.word_tokenize(txt)


words_dist = nltk.FreqDist(w for w in words if w not in stopwords+['england','russia','france','turkey','germany','italy','austria-hungary', 'austria'])

rslt = pd.DataFrame(words_dist.most_common(top_N),
                    columns=['Word', 'Frequency'])
display(rslt)

,Word,Frequency
0,thanks,48
1,thank,43
2,good,11
3,much,10
4,know,10
5,yes,10
6,great,8
7,help,8
8,forward,7
9,okay,7


### 3. doing the same analysis over VADER compond sentiments

* #### Looking at most negative messages

In [139]:
print('Most negative messages (according to VADER)')
for _, row in df.sort_values('vader_score').head(5).iterrows():
    print(f"Score: {row['vader_score']:.3f} | Message: {row['messages']}")

Most negative messages (according to VADER)
Score: -0.973 | Message: You're not, but you won't later either.  You can still hold Moscow with your fleet plus my army, and worst comes to worst, your army would be loose behind their front.  The alternative is we continue to attack you and we all lost out.  Italy, me, you
Score: -0.972 | Message: And I hate to be so greedy to open up but I'm nervous that if I forego a build it opens me up to have Russia, France or both collapse on me and then we're really fucked. Particularly if France walks into Munich, I'll need both builds badly
Score: -0.959 | Message: It depends: if they say "I don't trust Italy, I'm ok not going to Gal if you aren't", then you do blow some trust when you go to Gal.  BUT at the same time you went to Bla!  Austria and I commiserate around how much a dirty rat you are, you tell them that you were sure they were lying, and "sorry I didn't trust you, but I'll keep attacking Turkey, watch, they think I'll park my fleet in 

* #### Looking at first sentiment (quite negative) quartile messages

In [140]:
print('First quartile messages sample sorted by VADER sentiments')
for _, row in df.sort_values('vader_score').iloc[int(len(df)/4):,:].head(5).iterrows():
    print(f"Score: {row['vader_score']:.3f} | Message: {row['messages']}")

First quartile messages sample sorted by VADER sentiments
Score: 0.000 | Message: https://tenor.com/view/turkey-slice-cut-thanksgiving-gif-3575755
Score: 0.000 | Message: Understandable
Score: 0.000 | Message: I think I have something worked out that takes some of the issue off of you. You'll know it worked if you see some chairs in the West this turn. 😀
Score: 0.000 | Message: In my view, the fleet going to SC of Spain is a concession to England, but apparently he doesn't see things the same way. I suggest that we demilitarize Western Mediterranean for now. My fleet will go on standby in Lyon to defend Marseilles
Score: 0.000 | Message: Are you getting much from Austria?


* #### Looking at third sentiment (quite positive) quartile messages

In [141]:
print('Third quartile messages sample sorted by VADER sentiments')
for _, row in df.sort_values('vader_score').iloc[int(3*len(df)/4):,:].head(5).iterrows():
    print(f"Score: {row['vader_score']:.3f} | Message: {row['messages']}")

Third quartile messages sample sorted by VADER sentiments
Score: 0.511 | Message: We have trust
Score: 0.511 | Message: The Sev disband probably helped with the mutual trust 🙂
Score: 0.511 | Message: I’m with you 110%- the hun has grown strong
Score: 0.511 | Message: France is always strong early
Score: 0.511 | Message: That seems the most likely option, I think it's a good idea to do that. The French have the opportunity to ruin you completely if they manage to secure both.


* #### Looking at most positive messages

In [142]:
print('Most positive messages (according to VADER)')
for _, row in df.sort_values('vader_score', ascending=False).head(5).iterrows():
    print(f"Score: {row['vader_score']:.3f} | Message: {row['messages']}")

Most positive messages (according to VADER)
Score: 0.992 | Message: I agree, and I think we can safely consider France as doomed. I'm not saying England isn't sincere in that goal as well, but I'm trying to look at next year's problems. Or the year after that. When France is down and we're looking at what to do about Turkey - or England as the case may be. 

Probably if I sat on Portugal/Spain/MAO I could quite comfortably work with England, but that puts him into the position of going through you or northern Russia. Since you haven't tried to play me yet (or you're doing so well I haven't noticed yet lol), I know who I'll pick if I'm forced to.

Thanks! Yes I'm better now, just headachey. That sounds amazing, and I love the overnight together with friends. It sounds like so much fun, and a chance to catch up with her tribe before she marries into a new one :3
Score: 0.986 | Message: I just messaged Russia. I told them about your suggestion of a three-way alliance, and that I was mainl

### 4. Study of the vocabulary distribution

* #### most negative vocabulary distribution

In [143]:
top_N = 15
txt = df.sort_values('vader_score').head(100).messages.str.lower().str.replace(r'\|', ' ').str.cat(sep=' ')
txt = txt.translate(str.maketrans('', '', string.punctuation+'’´'))
words = nltk.tokenize.word_tokenize(txt)

words_dist = nltk.FreqDist(w for w in words if w not in stopwords+['england','russia','france','turkey','germany','italy','austria-hungary', 'austria'])

rslt = pd.DataFrame(words_dist.most_common(top_N),
                    columns=['Word', 'Frequency'])
display(rslt)

,Word,Frequency
0,im,40
1,get,30
2,stab,28
3,would,26
4,attack,25
5,fleet,20
6,could,20
7,dont,19
8,two,19
9,war,17


* #### quite negative vocabulary distribution (beginning of first quartile)

In [144]:
top_N = 10
txt = df.sort_values('vader_score').iloc[int(len(df)/4):,:].head(1000).messages.str.lower().str.replace(r'\|', ' ').str.cat(sep=' ')
txt = txt.translate(str.maketrans('', '', string.punctuation+'’´'))
words = nltk.tokenize.word_tokenize(txt)

words_dist = nltk.FreqDist(w for w in words if w not in stopwords+['england','russia','france','turkey','germany','italy','austria-hungary', 'austria'])

rslt = pd.DataFrame(words_dist.most_common(top_N),
                    columns=['Word', 'Frequency'])
display(rslt)

,Word,Frequency
0,think,76
1,f,74
2,im,68
3,going,56
4,move,49
5,get,42
6,ill,40
7,would,39
8,take,39
9,know,38


* #### quite positive vocabulary distribution (beginning of third quartile)

In [145]:
txt = df.sort_values('vader_score').iloc[int(3*len(df)/4):,:].head(1000).messages.str.lower().str.replace(r'\|', ' ').str.cat(sep=' ')
txt = txt.translate(str.maketrans('', '', string.punctuation+'’´'))
words = nltk.tokenize.word_tokenize(txt)

words_dist = nltk.FreqDist(w for w in words if w not in stopwords+['england','russia','france','turkey','germany','italy','austria-hungary', 'austria'])

rslt = pd.DataFrame(words_dist.most_common(top_N),
                    columns=['Word', 'Frequency'])
display(rslt)

,Word,Frequency
0,im,195
1,would,129
2,like,129
3,support,116
4,get,113
5,well,107
6,think,97
7,going,95
8,move,94
9,want,86


* #### very positive vocabulary distribution 

In [146]:
txt = df.sort_values('vader_score', ascending=False).head(100).messages.str.lower().str.replace(r'\|', ' ').str.cat(sep=' ')
txt = txt.translate(str.maketrans('', '', string.punctuation+'’´'))

# create a pattern that will match any country in pycountry.countries


# apply sub for each company
txt = pattern.sub("", txt).strip()

words = nltk.tokenize.word_tokenize(txt)

words_dist = nltk.FreqDist(w for w in words if w not in stopwords+['england','russia','france','turkey','germany','italy','austria-hungary', 'austria'])

rslt = pd.DataFrame(words_dist.most_common(top_N),
                    columns=['Word', 'Frequency'])
display(rslt)

,Word,Frequency
0,im,69
1,game,43
2,would,41
3,get,37
4,good,36
5,like,35
6,think,34
7,one,34
8,wintergreen,34
9,support,33
